## 如果是在Google Colab上執行，請先執行以下程式碼

In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

## 如果在本機執行，需先設定好`conda vars`變數，以取得各金鑰

In [3]:
import os
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
port = 5000

In [6]:
line_access_token = "iJZ84XlEcuKqdQEIx+xBY9v3U2GnHSAEZNiMuxkjwzXIYj+TG/teY5Va+/j9P0T4Q8EEyqlqH5F7n3RO78WQ0WXhzaKUM0czAlgpUeFq5jwif5fQgsaKhJceAD6o9B6u4dEGhbNhs67GkdRQWFkyRQdB04t89/1O/w1cDnyilFU="
line_secret = "9517d1ed64bd63d6512b5bd63198de1f"

In [7]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,    
)

from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                            TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/May/2025 23:50:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2025 23:50:05] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U4af8225b3ff2889a958695372a6ff69a","events":[]}
BODY:  {"destination":"U4af8225b3ff2889a958695372a6ff69a","events":[]}
